In [1]:
!pip install beautifulsoup4
!pip install -U textblob
!pip install nltk

Requirement already up-to-date: textblob in c:\users\vikas\anaconda3\lib\site-packages
Requirement already up-to-date: nltk>=3.1 in c:\users\vikas\anaconda3\lib\site-packages (from textblob)
Requirement already up-to-date: six in c:\users\vikas\anaconda3\lib\site-packages (from nltk>=3.1->textblob)


In [2]:
from bs4 import BeautifulSoup
import os,shutil,requests,csv
import pandas as pd
import numpy as np
import re
import textblob
import nltk
from textblob import TextBlob
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
def DownloadArticle():
    URL = r'http://www.thehindu.com/archive/print/2017/11/{}/'
    BOLD = '\033[1m';END = '\033[0m'
    DATES = [15,16,17,18,19]
    CITIES = ['New Delhi','Mumbai','Bengaluru','Chennai',
              'Hyderabad','Kolkata','Kochi','Vijayawada','Visakhapatnam']
    header = ['cityname','Article']
    print('data will be scrapped from 15th to 19th november')
    with open('Articles.csv','w',encoding='utf-8') as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow(header)
        for x in DATES:
            print(BOLD+'\ngetting articles for '+END + URL.format(x))
            resp = requests.get(URL.format(x))
            soup = BeautifulSoup(resp.content, 'lxml')
            for d in CITIES:
                count = 0
                print(BOLD + d + END,end='  ') #city
                for c in soup.find_all('h4',text=d.lower()):
                #print(c.text)

                    nex = c.find_next_sibling()
                    for UR in nex.find_all('a')[1:2]:
                        count+=1
                        article = []
                        ARTICLEURL = UR.get('href')
                        #print(ARTICLEURL)
                        res2 = requests.get(ARTICLEURL)
                        soup2 = BeautifulSoup(res2.content,'lxml')
                        for _ in soup2.find_all(['p'],_class=''):
                            article.append(_.text.strip())
                        try:
                            writer.writerow([d,article])
                        except:
                            pass
                #print(' - done with {} articles'.format(count))
    print('DONE..')
def transform(x):
    final = []

    stopwords = nltk.corpus.stopwords.words('english')
    _word = TextBlob(x)
    _word = _word.words.singularize().lemmatize()
    _word = [w for w in _word if w not in stopwords]
    return ' '.join(x for x in _word)

def analyse():
    df = pd.read_csv('Articles.csv')
    df['Article'] = df['Article'].apply(lambda x: re.sub(pattern = r'[\[*\]*\"*,*.*\n*\'**\?*\:*]',repl = ' ',string = x))
    df['Article'] = df.Article.apply(lambda x:transform(x))
    df['sentiment'] = df.Article.apply(lambda x: TextBlob(x).sentiment.polarity)
    finall = {}
    for x in df.cityname.values:
        finall[x] = (df.sentiment.loc[df.cityname==x].sum())/(df[df.cityname==x].shape[0])
    return finall

In [4]:
# run this if you want to download articles 
DownloadArticle()

data will be scrapped from 15th to 19th november

getting articles for http://www.thehindu.com/archive/print/2017/11/15/
New Delhi  Mumbai  Bengaluru  Chennai  Hyderabad  Kolkata  Kochi  Vijayawada  Visakhapatnam  
getting articles for http://www.thehindu.com/archive/print/2017/11/16/
New Delhi  Mumbai  Bengaluru  Chennai  Hyderabad  Kolkata  Kochi  Vijayawada  Visakhapatnam  
getting articles for http://www.thehindu.com/archive/print/2017/11/17/
New Delhi  Mumbai  Bengaluru  Chennai  Hyderabad  Kolkata  Kochi  Vijayawada  Visakhapatnam  
getting articles for http://www.thehindu.com/archive/print/2017/11/18/
New Delhi  Mumbai  Bengaluru  Chennai  Hyderabad  Kolkata  Kochi  Vijayawada  Visakhapatnam  
getting articles for http://www.thehindu.com/archive/print/2017/11/19/
New Delhi  Mumbai  Bengaluru  Chennai  Hyderabad  Kolkata  Kochi  Vijayawada  Visakhapatnam  DONE..


In [5]:
#run this if you wan to use downloaded csv file
analyse()

{'Bengaluru': 0.071024434771772713,
 'Chennai': 0.083531374897821017,
 'Hyderabad': 0.093896830965444184,
 'Kochi': 0.10830515645035063,
 'Kolkata': -0.028641345427059706,
 'Mumbai': 0.079088447967601172,
 'New Delhi': 0.040414718257335225,
 'Vijayawada': 0.11527100755450449,
 'Visakhapatnam': 0.1044379146128946}